In [17]:
pip install selenium


   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.1/9.6 MB 1.6 MB/s eta 0:00:06
    --------------------------------------- 0.2/9.6 MB 2.1 MB/s eta 0:00:05
   - -------------------------------------- 0.3/9.6 MB 2.6 MB/s eta 0:00:04
   - -------------------------------------- 0.4/9.6 MB 2.3 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/9.6 MB 2.7 MB/s eta 0:00:04
   --- ------------------------------------ 0.8/9.6 MB 3.0 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/9.6 MB 3.0 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/9.6 MB 3.6 MB/s eta 0:00:03
   ------ --------------------------------- 1.5/9.6 MB 3.6 MB/s eta 0:00:03
   ------ --------------------------------- 1.5/9.6 MB 3.3 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/9.6 MB 3.3 MB/s eta 0:00:03
   --------- ------------------------------ 2.3/9.6 MB 4.1 MB/s eta 0:00:02
   ----------- ----


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.Collecting webdriver-manager




[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Configuración de Selenium y Chrome
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Ejecuta en modo sin cabeza (sin abrir el navegador)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Base URL de las páginas de reseñas
base_url = "https://www.yelp.com/biz/m%C3%A9lisse-santa-monica-3?start="

# Lista para almacenar todas las reseñas
all_reviews = []

# Función para extraer reseñas de una página
def extract_reviews(url):
    driver.get(url)
    time.sleep(3)  # Espera para que se cargue la página
    reviews = []
    review_elements = driver.find_elements(By.CSS_SELECTOR, 'p.comment__09f24__D0cxf span.raw__09f24__T4Ezm')
    for review_element in review_elements:
        reviews.append({'review_text': review_element.text.strip()})
    return reviews

# Recorrer las páginas (cada página empieza en un múltiplo de 10)
for page in range(0, 1390, 10):  # 139 páginas, incrementando en 10
    url = base_url + str(page)
    print(f"Extrayendo reseñas de la página: {url}")
    page_reviews = extract_reviews(url)
    if not page_reviews:
        break  # Si no se encuentran reseñas, salir del bucle
    all_reviews.extend(page_reviews)
    time.sleep(2)  # Pausa de 2 segundos entre solicitudes

# Guardar los datos en un archivo CSV
df = pd.DataFrame(all_reviews)
df.to_csv('yelp_reviews_all.csv', index=False)
print("Datos guardados en 'yelp_reviews_all.csv'")

# Cerrar el navegador
driver.quit()


Extrayendo reseñas de la página: https://www.yelp.com/biz/m%C3%A9lisse-santa-monica-3?start=0
Extrayendo reseñas de la página: https://www.yelp.com/biz/m%C3%A9lisse-santa-monica-3?start=10
Extrayendo reseñas de la página: https://www.yelp.com/biz/m%C3%A9lisse-santa-monica-3?start=20
Extrayendo reseñas de la página: https://www.yelp.com/biz/m%C3%A9lisse-santa-monica-3?start=30
Extrayendo reseñas de la página: https://www.yelp.com/biz/m%C3%A9lisse-santa-monica-3?start=40
Extrayendo reseñas de la página: https://www.yelp.com/biz/m%C3%A9lisse-santa-monica-3?start=50
Extrayendo reseñas de la página: https://www.yelp.com/biz/m%C3%A9lisse-santa-monica-3?start=60
Extrayendo reseñas de la página: https://www.yelp.com/biz/m%C3%A9lisse-santa-monica-3?start=70
Extrayendo reseñas de la página: https://www.yelp.com/biz/m%C3%A9lisse-santa-monica-3?start=80
Extrayendo reseñas de la página: https://www.yelp.com/biz/m%C3%A9lisse-santa-monica-3?start=90
Extrayendo reseñas de la página: https://www.yelp.c

In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mi_cuenta_servicio.json'


In [3]:
from google.cloud import storage

def upload_to_gcs(file_name, bucket_name, destination_blob_name):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(file_name)
    print(f"Archivo {file_name} subido a {bucket_name}/{destination_blob_name}.")


In [5]:
bucket_name = 'scraping-bucket-uploader'  # Reemplaza con el nombre de tu bucket
csv_filename = 'yelp_reviews_all.csv'  # Nombre del archivo CSV
upload_to_gcs(csv_filename, bucket_name, csv_filename)
print("Proceso completado y archivo subido a Google Cloud Storage.")


Archivo yelp_reviews_all.csv subido a scraping-bucket-uploader/yelp_reviews_all.csv.
Proceso completado y archivo subido a Google Cloud Storage.
